In [2]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly_express as px
import pandas as pd
from dash import dash_table



In [3]:
stocks = pd.read_csv("../Data/stocks.csv", parse_dates=["Date"],index_col="Date")
stocks.head(2)

,Symbols,Close,High,Low,Open,Volume
Date,,,,,,
2023-01-03,AAPL,125.07,130.90,124.17,130.28,112117471
2023-01-03,MSFT,239.58,245.75,237.40,243.08,25740036


In [4]:
app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.FLATLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Stock Market Dashboard", className='text-center text-primary mx-3'),
            html.Br(),
            dcc.DatePickerRange(id='date_picker', className='mx-auto mb-4')
        ], width=12, style={'text-align': 'center'})
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id="single_dropdown", multi=False, searchable=False, className='mb-2', value='MSFT',
                options=[stock for stock in stocks["Symbols"].unique()],
                style={'color': '#333'}
            ),
            dcc.Graph(id="volume_graph", config={"staticPlot": True},
                figure={})
        ], xs=12, sm=11, md=10, lg=5),

        dbc.Col([
            dcc.Dropdown(
                id="multi_dropdown", multi=True, searchable=False, className='mb-2', value=['MSFT', 'AAPL'],
                options=[stock for stock in stocks["Symbols"].unique()],
                style={'color': '#333'}
            ),
            dcc.Graph(id="closing_graph", config={"staticPlot": True},
                figure={})
        ], xs=12, sm=11, md=10, lg=5)
    ], justify="evenly"),

    dbc.Row([
        dbc.Col([
            html.Br(),
            html.H2('MSFT', id='data_label'),
            dash_table.DataTable(id='data_table', page_size=15)
        ], width=10)
    ], justify='center'),


], fluid=True)

@callback(
    Output("volume_graph", "figure"),
    Output("data_table", "data"),
    Output("data_label", "children"),
    Input("date_picker", "start_date"),
    Input("date_picker", "end_date"),
    Input("single_dropdown", "value"),
    #State("data_label", "children")
)
def single_dropdown_changed(start, end, symbol): #, label):
    df = stocks.query("Symbols==@symbol").loc[start:end]
    fig = px.line(df, x=df.index, y="Volume")
    data = df.reset_index().drop(columns=["Symbols"]).to_dict('records')
    return fig, data, symbol #, f"{label}, {symbol}"

@callback(
    Output("closing_graph", "figure"),
    Input("date_picker", "start_date"),
    Input("date_picker", "end_date"),
    Input("multi_dropdown", "value")
)
def update_closing_graph(start, end, symbols):
    if symbols is None: symbols = []
    df = stocks[stocks["Symbols"].isin(symbols)].loc[start:end]
    return px.line(df, x=df.index, y="Close", color="Symbols")


app.run(debug=True, jupyter_mode='external')

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'